### Script #1
This notebook was developed to generate individual csv files for the flat top ethanol injection peaks measured by tag <code>FI6320.PV</code> in <code>akbm-houston-prod.houston_data.sensor_data_scada</code>. The thought behind this script was to use an algorithm to identify individual peaks, and them extract them.

To use this notebook successfully, the user will first have to identify regions where these nice regular peaks exist to identify a start time and and end time for the data series to be treated.

The goal is further to integrate these ethanol injection peaks using different numerical methods to confirm that amount of ethanol corresponds with the expected amount per extraction recipe for krill oil manufacturing.

In [ ]:
import pandas as pd
import pyodbc

# ======= Connect to Google BigQuery using ODBC =======
dsn = 'bq64_system'  # Your DSN
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)  # Ensure autocommit is enabled
cursor = conn.cursor()

# ======= User-defined global timestamps =======
global_start_time = "2025-01-18 00:00:01"  # Change this as needed
global_end_time = "2025-01-18 04:38:22"  # Change this as needed
tagname = "FI6320.PV"  # Change this if needed

# ======= Query to retrieve data from BigQuery =======
query = f"""
SELECT DISTINCT
  time AS timestamp,
  value AS value
FROM 
  `akbm-houston-prod.houston_data.sensor_data_scada`
WHERE 
  time BETWEEN TIMESTAMP('{global_start_time}') AND TIMESTAMP('{global_end_time}')
  AND time >= '1900-01-01' -- Partition filter for performance
  AND tagname = '{tagname}'
ORDER BY 
  timestamp;
"""

# Execute the query
cursor.execute(query)

# Fetch all results and convert from pyodbc.Row to a list of tuples
data = [tuple(row) for row in cursor.fetchall()]  # Ensure each row is a normal tuple

# Convert to DataFrame
df = pd.DataFrame(data, columns=[col[0] for col in cursor.description])

# Close the cursor and connection
cursor.close()
conn.close()

# Convert 'timestamp' column to datetime for proper time-based operations
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ======= Detect flat-top peaks =======
peak_threshold = 9000  # Adjust threshold as needed
df['is_peak'] = df['value'] > peak_threshold

# Group peaks based on continuous regions
df['peak_group'] = (df['is_peak'] != df['is_peak'].shift()).cumsum()
peak_groups = df[df['is_peak']].groupby('peak_group')

# Store detected peaks
refined_peak_segments = [group for _, group in peak_groups if len(group) > 5]

# ======= Identify baseline midpoints between peaks =======
baseline_midpoints = []
for i in range(len(refined_peak_segments) - 1):
    peak_end = refined_peak_segments[i]['timestamp'].max()
    next_peak_start = refined_peak_segments[i + 1]['timestamp'].min()
    midpoint = peak_end + (next_peak_start - peak_end) / 2  # Midpoint as separation
    baseline_midpoints.append(midpoint)

# ======= Define segment boundaries =======
global_start_time = df['timestamp'].min()
global_end_time = df['timestamp'].max()
segmentation_boundaries = [global_start_time] + baseline_midpoints + [global_end_time]

# Adjust segment starts so each starts at the **previous baseline midpoint** (not peak end)
segment_starts = [global_start_time] + baseline_midpoints[:-1]  # Start at previous baseline
segment_ends = baseline_midpoints + [global_end_time]  # End at next baseline

# ======= Extract individual segments and save to CSV =======
csv_files = []
for i, (start, end) in enumerate(zip(segment_starts, segment_ends)):
    segment_df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
    
    # Save segment to CSV
    file_name = f"segment_{i+1}.csv"
    segment_df.to_csv(file_name, index=False)
    csv_files.append(file_name)
    print(f"Saved: {file_name} ({len(segment_df)} rows)")

# ======= Extract final portion of data after last detected baseline =======
if baseline_midpoints:
    last_cutoff = baseline_midpoints[-1]  # Use the last detected midpoint
else:
    last_cutoff = segment_ends[-1]  # Fallback to the last segment end if no midpoints

final_segment_df = df[df['timestamp'] > last_cutoff]  # Ensure all remaining data is included
final_file_name = "segment_final.csv"
final_segment_df.to_csv(final_file_name, index=False)
csv_files.append(final_file_name)
print(f"Saved: {final_file_name} ({len(final_segment_df)} rows)")

print("\nAll segments successfully saved as CSV files.")



### Script #2 (currently the preferred solution. ignore script #1 for now)
##### Getting individual csv files for each extraction tank filling
This script was developed to generate individual csv files for individual ethanol injection peaks measured by tag <code>FI6320.PV</code> in <code>akbm-houston-prod.houston_data.sensor_data_scada</code>.<br>

To execute this script, the user provides an Excel list containing timestamps for the start and end times of each peak. The start and end times can, for example, be determined manually.

> v1: uses timestamps provided in Excel file, loops through them and gets the individual peaks <br>
> v2: performs baseline correction & adds a column showing time in seconds from start to end of each signal extracted.<br>
> The name of each csv file is the first time stamp in each csv file.

In [ ]:
# v1: no baseline correction and no "time passed" column showing time from 0 to end in seconds for each peak

import pyodbc
import pandas as pd
from datetime import datetime

# ======= Connect to Google BigQuery using ODBC =======
dsn = 'bq64_system'  # Your DSN
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)  # Ensure autocommit is enabled
cursor = conn.cursor()

# ======= Read start and end times from Excel =======
# Assumes that the Excel file "start & end times.xlsx" has no header and that:
#   Column 0 = start time, Column 1 = end time.
# If your Excel file has headers, adjust accordingly.
# timestamps are in format "15.05.2025  00:05:36" in each cell. This is UTC time as dd.mm.yyyy hh:mm:ss
time_ranges = pd.read_excel("timestamps for FI6320.xlsx", header=None)

tagname = "FI6320.PV"

# ======= Loop through each time range row =======
for idx, row in time_ranges.iterrows():
    # Terminate loop if both timestamp cells are missing (i.e. we've reached the end)
    if pd.isna(row[0]) and pd.isna(row[1]):
        print(f"No timestamp found in row {idx}. Terminating loop.")
        break

    # Convert Excel values to datetime objects; errors will raise if conversion fails
    start_time = pd.to_datetime(row[0], errors='raise')
    end_time = pd.to_datetime(row[1], errors='raise')
    
    # Format timestamps for the BigQuery query (YYYY-MM-DD HH:MM:SS format)
    start_str = start_time.strftime("%Y-%m-%d %H:%M:%S")
    end_str = end_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # Build the query using the current time range
    query = f"""
    SELECT DISTINCT
      time AS timestamp,
      value AS value
    FROM 
      `akbm-houston-prod.houston_data.sensor_data_scada`
    WHERE 
      time BETWEEN TIMESTAMP('{start_str}') AND TIMESTAMP('{end_str}')
      AND time >= '1900-01-01' -- Partition filter for performance
      AND tagname = '{tagname}'
    ORDER BY 
      timestamp;
    """
    
    # Execute the query
    cursor.execute(query)
    data = [tuple(r) for r in cursor.fetchall()]  # Convert each row to a normal tuple
    
    # Convert fetched data to a DataFrame
    df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])
    
    # Use the start time in 'dd-mm-yyyy-hh-mm' format for the CSV filename
    file_timestamp = start_time.strftime("%d-%m-%Y-%H-%M")
    csv_filename = f"data_series_{file_timestamp}.csv"
    
    # Export the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"Exported data series starting at {file_timestamp} to {csv_filename}")

# ======= Clean up =======
cursor.close()
conn.close()


In [ ]:
# v2: performs baseline correction & adds a column showing time in seconds from start to end of each signal extracted.

import pyodbc
import pandas as pd
from datetime import datetime

# ======= Connect to Google BigQuery using ODBC =======
dsn = 'bq64_system'  # Your DSN
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)  # Ensure autocommit is enabled
cursor = conn.cursor()

# ======= Read start and end times from Excel =======
# what the Excel file should look like
#      no headers
#      start time in column A (format: dd.mm.yyyy hh:mm:ss, example:11.05.2025  23:53:40)
#      end time in column B, same format as start time
time_ranges = pd.read_excel("timestamps for FI6320.xlsx", header=None)

tagname = "FI6320.PV"

# ======= Loop through each time range row =======
for idx, row in time_ranges.iterrows():
    # Terminate loop if both timestamp cells are missing (i.e. we've reached the end)
    if pd.isna(row[0]) and pd.isna(row[1]):
        print(f"No timestamp found in row {idx}. Terminating loop.")
        break

    # Convert Excel values to datetime objects; errors will raise if conversion fails
    start_time = pd.to_datetime(row[0], errors='raise')
    end_time = pd.to_datetime(row[1], errors='raise')
    
    # Format timestamps for the BigQuery query (YYYY-MM-DD HH:MM:SS format)
    start_str = start_time.strftime("%Y-%m-%d %H:%M:%S")
    end_str = end_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # Build the query using the current time range
    query = f"""
    SELECT DISTINCT
      time AS timestamp,
      value AS value
    FROM 
      `akbm-houston-prod.houston_data.sensor_data_scada`
    WHERE 
      time BETWEEN TIMESTAMP('{start_str}') AND TIMESTAMP('{end_str}')
      AND time >= '1900-01-01' -- Partition filter for performance
      AND tagname = '{tagname}'
    ORDER BY 
      timestamp;
    """
    
    # Execute the query
    cursor.execute(query)
    data = [tuple(r) for r in cursor.fetchall()]  # Convert each row to a normal tuple
    
    # Convert fetched data to a DataFrame
    df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])
    
    # Skip CSV export if no data is returned
    if df.empty:
        print(f"No data returned for time range starting {start_str}. Skipping CSV export.")
        continue
    
    # Modification 1: Baseline correction
    # Create a new column that subtracts 1.71578 from "value"
    df['value_corrected'] = df['value'] - 1.71578
    
    # Modification 2: Add a column for cumulative seconds passed
    # Convert the "timestamp" column to datetime
    df['timestamp_dt'] = pd.to_datetime(df['timestamp'])
    # Compute the time difference (in seconds) between consecutive timestamps and perform a cumulative sum
    df['seconds_passed'] = df['timestamp_dt'].diff().dt.total_seconds().fillna(0).cumsum()
    # Drop the intermediate datetime column if not needed
    df.drop(columns=['timestamp_dt'], inplace=True)
    
    # Use the start time in 'dd-mm-yyyy-hh-mm' format for the CSV filename
    file_timestamp = start_time.strftime("%d-%m-%Y-%H-%M")
    csv_filename = f"data_series_{file_timestamp}.csv"
    
    # Export the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"Exported data series starting at {file_timestamp} to {csv_filename}")

# ======= Clean up =======
cursor.close()
conn.close()


### Script 3
#### Signal integration & aggregation
Integrating aggregating the area under the curve for each csv file.<br>

* <b>Trapezoidal rule & Rieman are "time aware" integration methods, giving consideration to the fact that timepoints are not evenly spaced.</b><br>
* Simpson's rule is more sensitive to uneven time grids, but the results between the three methods are all in relatively good agreement.<br>
* Aggregation (simple summation) is sensitive to the spacing of the time steps. **Resampling with interpolation between missing data to fill the gaps can be used to compensate**.<br>

<u>The follwing results are given by the script below in the exported csv:
| Metric Name                                      | Explanation |
|--------------------------------------------------|-------------|
| **Original data results:**                       |             |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_trapezoid_value       | from original data, without resampling, no baseline correction             |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_simpson_value         | from original data, without resampling, no baseline correction           |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_riemann_value         | from original data, without resampling, no baseline correction            |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_trapezoid_value_corr  | from original data, without resampling, baseline corrected            |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_simpson_value_corr    | from original data, without resampling, baseline corrected            |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_riemann_value_corr    | from original data, without resampling, baseline corrected            |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_sum_value             | aggregation, from original data, without resampling, no baseline correction            |
| &nbsp;&nbsp;&nbsp;&nbsp;orig_sum_value_corr        | aggregation, from original data, without resampling, baseline corrected            |
| **Resampled data results:**                      |             |
| &nbsp;&nbsp;&nbsp;&nbsp;res_trapezoid_value        | from resampled data, no baseline correction           |
| &nbsp;&nbsp;&nbsp;&nbsp;res_simpson_value          | from resampled data, no baseline correction            |
| &nbsp;&nbsp;&nbsp;&nbsp;res_riemann_value          | from resampled data, no baseline correction            |
| &nbsp;&nbsp;&nbsp;&nbsp;res_trapezoid_value_corr     | from resampled data, baseline corrected             |
| &nbsp;&nbsp;&nbsp;&nbsp;res_simpson_value_corr       | from resampled data, baseline corrected            |
| &nbsp;&nbsp;&nbsp;&nbsp;res_riemann_value_corr       | from resampled data, baseline corrected            |
| &nbsp;&nbsp;&nbsp;&nbsp;res_sum_value              | aggergation, from resampled data, no baseline correction             |
| &nbsp;&nbsp;&nbsp;&nbsp;res_sum_value_corr         | aggergation, from resampled data, baseline corrected           |



In [ ]:
import pandas as pd
import numpy as np
import glob
from scipy.integrate import simpson  # Make sure you have scipy installed

# Set the directory where your CSV files are located
directory_path = "peaks"  # e.g., "data" or "./"

# Pattern to match your CSV files
csv_files = glob.glob(f"{directory_path}/data_series_*.csv")

# List to store summary results
summary_results = []

for file in csv_files:
    # ---------------------------
    # ORIGINAL DATA CALCULATIONS
    # ---------------------------
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Convert "timestamp" column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # If "seconds_passed" is not in the DataFrame, compute it
    if "seconds_passed" not in df.columns:
        df['seconds_passed'] = df['timestamp'].diff().dt.total_seconds().fillna(0).cumsum()
    
    # Prepare arrays for original data integration
    x_orig = df['seconds_passed'].values
    y_value_orig = df['value'].values
    y_value_corr_orig = df['value_corrected'].values
    
    # Integration on original data:
    # 1. Trapezoidal rule
    orig_trap_value = np.trapezoid(y_value_orig, x_orig) / 3600
    orig_trap_value_corr = np.trapezoid(y_value_corr_orig, x_orig) / 3600
    
    # 2. Simpson's rule
    try:
        orig_simpson_value = simpson(y_value_orig, x_orig) / 3600
    except Exception as e:
        orig_simpson_value = np.nan
        print(f"Simpson integration error for {file} on original 'value': {e}")
    try:
        orig_simpson_value_corr = simpson(y_value_corr_orig, x_orig) / 3600
    except Exception as e:
        orig_simpson_value_corr = np.nan
        print(f"Simpson integration error for {file} on original 'value_corrected': {e}")
    
    # 3. Riemann sum (left endpoint)
    orig_riemann_value = np.sum(y_value_orig[:-1] * np.diff(x_orig)) / 3600
    orig_riemann_value_corr = np.sum(y_value_corr_orig[:-1] * np.diff(x_orig)) / 3600
    
    # Aggregation (simple sum) for original data
    orig_sum_value = np.sum(y_value_orig) / 3600
    orig_sum_value_corr = np.sum(y_value_corr_orig) / 3600
    
    # ----------------------------
    # RESAMPLED DATA CALCULATIONS
    # ----------------------------
    # Create a copy and set timestamp as index
    df_resample = df.copy()
    df_resample.set_index('timestamp', inplace=True)
    
    # Resample the data to a fixed 1-second frequency and interpolate missing values
    resampled_df = df_resample.resample('1s').mean().interpolate('linear')
    
    # Create a "seconds_passed" column based on the resampled index
    resampled_df['seconds_passed'] = (resampled_df.index - resampled_df.index[0]).total_seconds()
    
    # Prepare arrays for resampled data integration
    x_res = resampled_df['seconds_passed'].values
    y_value_res = resampled_df['value'].values
    y_value_corr_res = resampled_df['value_corrected'].values
    
    # Integration on resampled data:
    # 1. Trapezoidal rule
    res_trap_value = np.trapezoid(y_value_res, x_res) / 3600
    res_trap_value_corr = np.trapezoid(y_value_corr_res, x_res) / 3600
    
    # 2. Simpson's rule
    try:
        res_simpson_value = simpson(y_value_res, x_res) / 3600
    except Exception as e:
        res_simpson_value = np.nan
        print(f"Simpson integration error for {file} on resampled 'value': {e}")
    try:
        res_simpson_value_corr = simpson(y_value_corr_res, x_res) / 3600
    except Exception as e:
        res_simpson_value_corr = np.nan
        print(f"Simpson integration error for {file} on resampled 'value_corrected': {e}")
    
    # 3. Riemann sum (left endpoint)
    res_riemann_value = np.sum(y_value_res[:-1] * np.diff(x_res)) / 3600
    res_riemann_value_corr = np.sum(y_value_corr_res[:-1] * np.diff(x_res)) / 3600
    
    # Aggregation (simple sum) for resampled data
    res_sum_value = np.sum(y_value_res) / 3600
    res_sum_value_corr = np.sum(y_value_corr_res) / 3600
    
    # Get the first timestamp (for reference)
    first_timestamp = df['timestamp'].iloc[0].strftime("%Y-%m-%d %H:%M:%S")
    
    # Append results for both original and resampled data to the summary list
    summary_results.append({
        'filename': file,
        'first_timestamp': first_timestamp,
        # Original data results:
        'orig_trapezoid_value': orig_trap_value,
        'orig_simpson_value': orig_simpson_value,
        'orig_riemann_value': orig_riemann_value,
        'orig_trapezoid_value_corr': orig_trap_value_corr,
        'orig_simpson_value_corr': orig_simpson_value_corr,
        'orig_riemann_value_corr': orig_riemann_value_corr,
        'orig_sum_value': orig_sum_value,
        'orig_sum_value_corr': orig_sum_value_corr,
        # Resampled data results:
        'res_trapezoid_value': res_trap_value,
        'res_simpson_value': res_simpson_value,
        'res_riemann_value': res_riemann_value,
        'res_trapezoid_value_corr': res_trap_value_corr,
        'res_simpson_value_corr': res_simpson_value_corr,
        'res_riemann_value_corr': res_riemann_value_corr,
        'res_sum_value': res_sum_value,
        'res_sum_value_corr': res_sum_value_corr,
    })

# Create a summary DataFrame and print results (no new CSV files are created)
summary_df = pd.DataFrame(summary_results)
print(summary_df)

# Save the summary DataFrame to a CSV file
summary_df.to_csv("integration_summary.csv", index=False)
print("Integration summary saved to integration_summary.csv")
